# Data Cleaning & Feature Engineering
As the datasets provided are messy, this is very helpful to clean it before loading into database and for that python and pandas comes in power. Hence this notebook will step by step clean the given data to get it ready for loading into a database and running SQL quaeries.

## Load necessary libraries and Initial setup

In [57]:
# Load libraries
import pandas as pd
import numpy as np
import re
import hashlib

# Set file path
path = "../data/"

## Load raw data files

In [58]:
# Load data files
games_raw = pd.read_csv(path + "games.csv")
sales_raw = pd.read_csv(path + "vgsales.csv")

## Inspect data

### Shape & Size

In [59]:
print("games.csv shape: ", games_raw.shape)
print("vgsales.csv shape: ", sales_raw.shape)

games.csv shape:  (1512, 14)
vgsales.csv shape:  (16598, 11)


### Column names & data types

In [60]:
games_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1512 non-null   int64  
 1   Title              1512 non-null   object 
 2   Release Date       1512 non-null   object 
 3   Team               1511 non-null   object 
 4   Rating             1499 non-null   float64
 5   Times Listed       1512 non-null   object 
 6   Number of Reviews  1512 non-null   object 
 7   Genres             1512 non-null   object 
 8   Summary            1511 non-null   object 
 9   Reviews            1512 non-null   object 
 10  Plays              1512 non-null   object 
 11  Playing            1512 non-null   object 
 12  Backlogs           1512 non-null   object 
 13  Wishlist           1512 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 165.5+ KB


In [61]:
sales_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


### Preview Data

In [62]:
games_raw.head()

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,3,Undertale,"Sep 15, 2015","['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,4,Hollow Knight,"Feb 24, 2017",['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K


In [63]:
sales_raw.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Check Missing Values

In [64]:
games_raw.isna().sum().sort_values(ascending=False)

Rating               13
Team                  1
Summary               1
Title                 0
Release Date          0
Unnamed: 0            0
Times Listed          0
Number of Reviews     0
Genres                0
Reviews               0
Plays                 0
Playing               0
Backlogs              0
Wishlist              0
dtype: int64

In [65]:
sales_raw.isna().sum().sort_values(ascending=False)

Year            271
Publisher        58
Rank              0
Platform          0
Name              0
Genre             0
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

### Check Uniqueness & Join Readiness

In [66]:
games_raw['Title'].nunique(), games_raw.shape[0]

(1099, 1512)

In [67]:
sales_raw['Name'].nunique(), sales_raw.shape[0]

(11493, 16598)

### Key Observations & Issues
1. Engagement metrics are are stored as text (e.g., "1.2K", "3M") making them unsuitable for numerical analysis.
2. Multivalued categorical fields (e.g., 'categories', 'tags') are stored as comma-separated strings, complicating filtering and aggregation.
3. Missing platform information in the engagement data hinders cross-referencing.
4. Year stored as float with missing values.
5. Extra index column in engagement data.
6. One-to-many relationships between datasets need to be addressed for proper joins.
7. Many titles contains roman and numerical values for version info.
8. There are many duplicate records in both datasets which needs to be removed.

## Data Cleaning and Transformation
### Clean Game Engagement Data
#### Drop Unnecessary Columns
As we can see from the above inspection, there are some columns that are not useful for our analysis and can be dropped to reduce clutter and improve performance. We will drop the following columns:
- Unnamed: 0 (index column)
- Summary (text data that is not useful for our analysis)
- Reviews (qualitative text data that is not useful for our analysis)

In [68]:
games = games_raw.drop(
    columns=[
        "Unnamed: 0",
        "Summary",
        "Reviews"
    ],
    errors="ignore"
)

#### Standardize column names
To ensure consistency and ease of use, we will standardize the column names by:
- removing spaces and special characters
- converting to lowercase
- using underscores to separate words

In [69]:
games.columns = (
    games.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

#### Convert Engagement Metrics to Numeric
For better operability, let's define a function to convert the engagement metrics from text to numeric values.

In [70]:
def parse_numeric(val):
    if pd.isna(val):
        return np.nan
    val = str(val).upper().replace(",", "").strip()
    multipliers = {"K": 1e3, "M": 1e6, "B": 1e9}
    if val[-1] in multipliers:
        try:
            return float(val[:-1]) * multipliers[val[-1]]
        except ValueError:
            return np.nan
    try:
        return float(val)
    except ValueError:
        return np.nan

Now, set a list of engagement metric columns to be converted and apply the conversion function to each of them.

In [71]:
numeric_cols = [
    "times_listed",
    "number_of_reviews",
    "plays",
    "playing",
    "backlogs",
    "wishlist",
]

for col in numeric_cols:
    games[col] = games[col].apply(parse_numeric)

#### Date Conversion
Convert the 'release_date' column to datetime format for easier date-based analysis.

In [72]:
games['release_date'] = pd.to_datetime(
    games['release_date'],
    errors='coerce'
)

#### Convert Ratings
Convert the 'rating' column to numeric format, handling any non-numeric entries appropriately.

In [73]:
games['rating'] = pd.to_numeric(
    games['rating'],
    errors='coerce'
)

#### Fix titles with Roman and Numerical Version Info
Many game titles contain version information in Roman numerals ornumerical format. We need to standardize these titles to ensure consistency across datasets.

In [74]:
roman_map = {
    "i": "1",
    "ii": "2",
    "iii": "3",
    "iv": "4",
    "v": "5",
    "vi": "6",
    "vii": "7",
    "viii": "8",
    "ix": "9",
    "x": "10"
}

def normalize_title(title):
    title = str(title).strip().lower()
    title = re.sub(r'[:\-_,]', ' ', title)  # Remove punctuation

    for roman, num in roman_map.items():
        pattern = r'\b' + re.escape(roman) + r'\b'
        title = re.sub(pattern, num, title)

    title = re.sub(r'\s+', ' ', title).strip()  # Replace multiple spaces with single space
    return title

games['normalized_title'] = games['title'].apply(normalize_title)

#### Remove Duplicates
To ensure data integrity and avoid skewed analysis, we will remove any duplicate records based on title and release date columns.

In [75]:
# remove duplicates based on normalized title and release date
games = games.drop_duplicates(subset=['normalized_title', 'release_date'], keep='first').reset_index(drop=True)

#### Create a Unique Identifier
To facilitate easier joins and analysis, we will create a unique identifier for each game by hashing the combination of title and release date.

In [76]:
# function to generate unique key
def generate_key(title, year):
    key_str = f"{title}_{year}"
    return hashlib.md5(key_str.encode()).hexdigest()

# create unique identifier for games dataset
games['id'] = games.apply(
    lambda row: generate_key(
        row['normalized_title'],
        row['release_date'].year if pd.notna(row['release_date']) else ''
    ),
    axis=1
)

#### Normalize Generes
The 'genres' column contains multiple genres in a single string, separated by commas. To enable better analysis and filtering, we will normalize this column by splitting the genres into separate rows.

In [77]:
import ast
game_genres = (
    games[['id', 'genres']]
    .dropna(subset=['genres'])
    .assign(genres=lambda x: x['genres'].apply(ast.literal_eval))
    .explode('genres')
)
game_genres['genre'] = game_genres['genres'].str.strip()
game_genres = game_genres[['id', 'genre']]

#### Normalize Developers
Similar to genres, the 'developers' column also contains multiple developers in a single string. We will normalize this column by splitting the developers into separate rows.

In [78]:
game_teams = (
    games[['id', 'team']]
    .dropna()
    .assign(team = lambda x : x['team'].apply(ast.literal_eval))
    .explode('team')
)

game_teams['developer'] = game_teams['team'].str.strip()
game_teams = game_teams[['id', 'developer']]

#### Remove Unnecessary Columns and Rearrange Columns
After the necessary transformations, we can drop any columns that are no longer needed for our analysis to reduce clutter and improve performance.

In [79]:
games_clean = games.drop(columns=['title', 'genres', 'team'], errors='ignore')
games_clean = games_clean.rename(columns={'normalized_title': 'title'})
games_clean = games_clean[[
    'id',
    'title',
    'release_date',
    'rating',
    'times_listed',
    'number_of_reviews',
    'plays',
    'playing',
    'backlogs',
    'wishlist'
]]

### Clean Game Sales Data
#### Standardize column names
To ensure consistency and ease of use, we will standardize the column names by:
- removing spaces and special characters
- converting to lowercase
- using underscores to separate words

In [80]:
sales = sales_raw.copy()

sales.columns = (
    sales.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

#### Normalize titles with Roman and Numerical Version Info
Similar to the game engagement data, we will standardize the titles in the game sales data to ensure consistency across datasets.

In [81]:
sales['normalized_name'] = sales['name'].apply(normalize_title)

#### Fix Year Data
The 'year' column contains some missing values and is stored as a float. We will convert it to an integer and handle missing values appropriately.

In [82]:
sales['year'] = pd.to_numeric(sales['year'], errors='coerce').astype('Int64')

# Find missing values in 'year' column
missing_years = sales[sales['year'].isna()]

# Fill missing years from sales data if possible
for idx, row in missing_years.iterrows():
    game_name = row['normalized_name'].strip().lower()
    possible_years = games_clean[
        games_clean['title'].str.strip().str.lower() == game_name]['release_date'].dropna().dt.year.unique()
    if len(possible_years) == 1:
        sales.at[idx, 'year'] = possible_years[0]

missing_years = sales[sales['year'].isna()]
print(f"Number of records with missing year: {missing_years.shape[0]}")

Number of records with missing year: 260


#### Remove Duplicates
To ensure data integrity and avoid skewed analysis, we will remove any duplicate records based on the name, platform, and year columns.

In [83]:
# Remove duplicates based on normalized name, platform and year
sales = sales.drop_duplicates(subset=['normalized_name', 'platform', 'year'], keep='first').reset_index(drop=True)

#### Create a Unique Identifier
To facilitate easier joins and analysis, we will create a unique identifier for each game by hashing the combination of title and release year

In [84]:
# create unique identifier for sales dataset
sales['id'] = sales.apply(
    lambda row: generate_key(
        row['normalized_name'],
        row['year'] if pd.notna(row['year']) else ''
    ),
    axis=1
)

#### Drop Unnecessary Columns and Rearrange Columns
After the necessary transformations, we can drop any columns that are no longer needed for our analysis to reduce clutter and improve performance.

In [85]:
sales_clean = sales.drop(columns=['name'], errors='ignore')
sales_clean = sales_clean.rename(
    columns= {'normalized_name':'name'}
)
sales_clean = sales_clean[[
    'id',
    'name',
    'platform',
    'year',
    'genre',
    'publisher',
    'na_sales',
    'eu_sales',
    'jp_sales',
    'other_sales',
    'global_sales'
]]

In [89]:
print(games_clean[games_clean.duplicated(subset=['id'], keep=False)])
print(sales_clean['id'].duplicated().sum())

                                    id                 title release_date  \
236   108e4f3f4095310d587ad222b53b0ec6                tetris   1989-11-01   
570   6ca39185135084c35a40a11eac933e77  sonic the hedgehog 2   1992-11-21   
828   108e4f3f4095310d587ad222b53b0ec6                tetris   1989-05-14   
1098  6ca39185135084c35a40a11eac933e77  sonic the hedgehog 2   1992-10-16   

      rating  times_listed  number_of_reviews    plays  playing  backlogs  \
236      4.1         688.0              688.0   9600.0     54.0     175.0   
570      3.7        1200.0             1200.0  10000.0     56.0     883.0   
828      4.0         197.0              197.0   2500.0     18.0      76.0   
1098     2.8         157.0              157.0   1400.0      6.0     173.0   

      wishlist  
236       66.0  
570      228.0  
828       21.0  
1098      45.0  
4253


## Export Cleaned Data

In [87]:
games_clean.to_csv(path + "games_clean.csv", index=False)
game_genres.to_csv(path + "game_genres.csv", index=False)
game_teams.to_csv(path + "game_teams.csv", index=False)
sales_clean.to_csv(path + "sales_clean.csv", index=False)